In [7]:
import torch

from model.vocabulary import vocabulary_answers as target_vocabulary
from model.vocabulary import vocabulary_expressions as source_vocabulary

import json

In [8]:
from model.model import Model

In [9]:
config_path = "./configs/default.json"
print(f"Using configuration: {config_path}")

file = open(config_path, "r")
config = json.load(file)
file.close()

Using configuration: ./configs/default.json


# Load model from checkpoint

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
model = Model(
    source_vocab_size=len(source_vocabulary), source_embedding_size=config["embedding_size"],
    target_vocab_size=len(target_vocabulary), target_embedding_size=config["embedding_size"],
    encoding_size=config["rnn_hidden_size"], target_bos_index=source_vocabulary.begin_seq_index,
    max_seq_length=dataset.max_seq_length_target
).to(device)

checkpoint = torch.load("model.pth")
model.load_state_dict(checkpoint['model_state_dict'])
model

TypeError: Model.__init__() missing 7 required positional arguments: 'source_vocab_size', 'source_embedding_size', 'target_vocab_size', 'target_embedding_size', 'encoding_size', 'target_bos_index', and 'max_seq_length'

In [ ]:
model.eval()

# Helper functions

In [5]:
def to_indices(scores):
    _, indices = torch.max(scores, dim=1)
    return indices

def sentence_from_indices(indices, vocab, strict=True):
    out = []
    for index in indices:
        index = index.item()
        if index == vocab.begin_seq_index and strict:
            continue
        elif index == vocab.end_seq_index and strict:
            return " ".join(out)
        else:
            out.append(vocab.getToken(index))
    return " ".join(out)

# Inference

In [ ]:
# n^2 test
test_expression = ["#", "/", "0", "0"]
test_tensor = torch.tensor([
    vocabulary_expressions.vectorize(test_expression) for _ in range(config["batch_size"])
], dtype=torch.int32)
test_pred = model(
    test_tensor,
    torch.LongTensor([len(test_tensor[0]) for _ in range(len(test_tensor))]),
    target_sequence=None
)

print(f"Test expression: {test_expression}")
print(f"Predicted shape: {test_pred.shape}")
print(f"Predicted value: {sentence_from_indices(to_indices(test_pred[0]), vocabulary_answers)}")

UndefVarError: UndefVarError: config not defined